<a href="https://colab.research.google.com/github/AdarshShah/final-year-project/blob/master/project_sepsis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Libraries

In [0]:
import time
import pandas as pd
import psycopg2
import getpass
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Model, Sequential
from keras.layers import Input, GRU, Dense, LSTM, Flatten

# Declare Global Variables

In [0]:
user = 'postgres'
host = 'localhost'
dbname = 'mimic'
schema = 'mimiciii'
password = 'postgres'
icd9_code = '99592'
minMaxScaler = dict()
itemMeans = dict()

# Create Database Connection

In [0]:
conn = psycopg2.connect(user=user,password=password,dbname=dbname,host=host)
cur = conn.cursor()
cur.execute('SET search_path to {}'.format(schema))

# Load Patient data




> Data from admissions suffering from severe sepsis

In [0]:
query = """SELECT DISTINCT * FROM mimiciii.admissions i 
INNER JOIN mimiciii.diagnoses_icd USING(hadm_id)
INNER JOIN mimiciii.d_icd_diagnoses USING(icd9_code)
WHERE icd9_code LIKE '99592';
"""
admissions = pd.read_sql_query(sql=query,con=conn)
admissions['sepsis'] = 1

query = """SELECT DISTINCT * FROM mimiciii.admissions i 
INNER JOIN mimiciii.diagnoses_icd USING(hadm_id)
INNER JOIN mimiciii.d_icd_diagnoses USING(icd9_code)
WHERE icd9_code NOT LIKE '99592' LIMIT 4000;
"""
temp = pd.read_sql_query(sql=query,con=conn)
temp['sepsis'] = 0

admissions = admissions.append(temp)



> List Items from d_labitems and d_items

In [0]:
#d_items = ['Platelets','C Reactive Protein (CRP)','pH (Art)','HCO3','TCO2 (cap)']
#d_labitems = ['Leukocytes','Urea Nitrogen','Creatinine','Glucose','Calcium, Total','Sodium','Potassium','Bilirubin','Albumin','Lactate','pCO2','pO2','Hemoglobin']

#Append itemid below !!!!
d_item_ce = ['0','828','3789','6256','791','3750','1525','220615','811','3744','3745','1529','226537','837','3803','1536','4948','3066','227444','3835','3837','812','3736']  
d_item_cv = ['0','30006','44711','44441']
d_item_mv = ['0','225170']
d_labitem = ['51486','51006','50912','50809','50931','51478','50813','50820']

In [0]:
def get_pandas(items, table):
  query = "SELECT DISTINCT * FROM mimiciii."+table+" WHERE itemid IN " + str(tuple(items))
  items = pd.read_sql_query(sql=query,con=conn)
  return items

d_item_ce = get_pandas(d_item_ce,"d_items")
d_item_cv = get_pandas(d_item_cv,"d_items")
d_item_mv = get_pandas(d_item_mv,"d_items")
d_labitem = get_pandas(d_labitem,"d_labitems")

# Fetch items from mimic-iii

> CHARTEVENTS

In [0]:
query = "SELECT * FROM mimiciii.chartevents WHERE itemid IN " + str(tuple(d_item_ce['itemid'])) + " AND hadm_id IN " + str(tuple(admissions['hadm_id'])) + " ORDER BY hadm_id,charttime,itemid;"
chartevents = pd.read_sql_query(sql=query,con=conn)
chartevents['row_id'] = range(100,100+chartevents['row_id'].count())

In [0]:
itemids = chartevents['itemid'].drop_duplicates()
temp = chartevents.pivot(index='row_id',columns='itemid',values='valuenum')
temp.fillna(value=temp.mean(skipna=True),inplace=True)
for item in itemids:
  arr = np.reshape(np.array(temp[item]),(-1,1))
  minMaxScaler[item] = MinMaxScaler().fit(arr)

In [0]:
temp = dict(temp.mean())
for key in temp.keys():
  itemMeans[key] = temp[key]

> INPUTEVENTS_CV



In [0]:
query = "SELECT * FROM mimiciii.inputevents_cv WHERE itemid IN " + str(tuple(d_item_cv['itemid'])) + " AND hadm_id IN " + str(tuple(admissions['hadm_id'])) + " ORDER BY hadm_id,charttime,itemid;"
inputevents_cv = pd.read_sql_query(sql=query,con=conn)
inputevents_cv['row_id'] = range(100,100+inputevents_cv['row_id'].count())

In [0]:
itemids = inputevents_cv['itemid'].drop_duplicates()
temp = inputevents_cv.pivot(index='row_id',columns='itemid',values='amount')
temp.fillna(value=temp.mean(skipna=True),inplace=True)
for item in itemids:
  arr = np.reshape(np.array(temp[item]),(-1,1))
  minMaxScaler[item] = MinMaxScaler().fit(arr)

In [0]:
temp = dict(temp.mean())
for key in temp.keys():
  itemMeans[key] = temp[key]

> INPUTEVENTS_MV

In [0]:
query = "SELECT * FROM mimiciii.inputevents_mv WHERE itemid IN ('225170') AND hadm_id IN " + str(tuple(admissions['hadm_id'])) + " ORDER BY hadm_id,starttime,itemid;"
inputevents_mv = pd.read_sql_query(sql=query,con=conn)
inputevents_mv['row_id'] = range(100,100+inputevents_mv['row_id'].count())

In [0]:
itemids = inputevents_mv['itemid'].drop_duplicates()
temp = inputevents_mv.pivot(index='row_id',columns='itemid',values='amount')
temp.fillna(value=temp.mean(skipna=True),inplace=True)
for item in itemids:
  arr = np.reshape(np.array(temp[item]),(-1,1))
  minMaxScaler[item] = MinMaxScaler().fit(arr)

In [0]:
temp = dict(temp.mean())
for key in temp.keys():
  itemMeans[key] = temp[key]

> LABEVENTS

In [0]:
query = "SELECT * FROM mimiciii.labevents WHERE itemid IN " + str(tuple(d_labitem['itemid'])) + " AND hadm_id IN " + str(tuple(admissions['hadm_id'])) + " ORDER BY hadm_id,charttime,itemid;"
labevents = pd.read_sql_query(sql=query,con=conn)
labevents['row_id'] = range(100,100+labevents['row_id'].count())

In [0]:
itemids = labevents['itemid'].drop_duplicates()
temp = labevents.pivot(index='row_id',columns='itemid',values='valuenum')
temp.fillna(value=temp.mean(skipna=True),inplace=True)
for item in itemids:
  arr = np.reshape(np.array(temp[item]),(-1,1))
  minMaxScaler[item] = MinMaxScaler().fit(arr)

In [0]:
temp = dict(temp.mean())
for key in temp.keys():
  itemMeans[key] = temp[key]

# Generate Tensors

In [0]:
hadm_id = admissions['hadm_id']

In [0]:
x_sepsis_dataset = []
y_sepsis_dataset = []
for hadm in hadm_id:
  data = pd.DataFrame(data=itemMeans,index={0})

  temp = chartevents[chartevents['hadm_id'] == hadm]
  temp = temp.pivot(index='row_id',columns='itemid',values='valuenum')
  data = data.append(temp,ignore_index=False)
  
  temp = inputevents_cv[inputevents_cv['hadm_id'] == hadm]
  temp = temp.pivot(index='row_id',columns='itemid',values='amount')
  data = data.append(temp,ignore_index=False)
  
  temp = inputevents_mv[inputevents_mv['hadm_id'] == hadm]
  temp = temp.pivot(index='row_id',columns='itemid',values='amount')
  data = data.append(temp,ignore_index=False)

  temp = labevents[labevents['hadm_id'] == hadm]
  temp = temp.pivot(index='row_id',columns='itemid',values='valuenum')
  data = data.append(temp,ignore_index=False)

  # mean normalization
  data.fillna(value=itemMeans,inplace=True)
  for key in minMaxScaler.keys():
    arr = np.reshape(np.array(data[key]),(-1,1))
    data[key] = minMaxScaler[key].transform(arr)

  y_sepsis_dataset.insert(len(y_sepsis_dataset),int(admissions[admissions['hadm_id']==hadm]['sepsis'].all()))
  x_sepsis_dataset.insert(len(x_sepsis_dataset),np.array(data))

In [0]:
x_sepsis_dataset = np.array(x_sepsis_dataset)
y_sepsis_dataset = np.array(y_sepsis_dataset)

> Train Test Split

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x_sepsis_dataset,y_sepsis_dataset,test_size=0.20,shuffle=True)

In [174]:
np.shape(x_train)

(6329,)

# Model

In [193]:
sepsis = Sequential()
sepsis.add(LSTM(128,input_shape=(None,31),return_sequences=True))
sepsis.add(LSTM(32,return_sequences=True))
sepsis.add(LSTM(8,return_sequences=False))
sepsis.add(Dense(8,activation='sigmoid'))
sepsis.add(Dense(1,activation='sigmoid'))

sepsis.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
sepsis.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_43 (LSTM)               (None, None, 128)         81920     
_________________________________________________________________
lstm_44 (LSTM)               (None, None, 32)          20608     
_________________________________________________________________
lstm_45 (LSTM)               (None, 8)                 1312      
_________________________________________________________________
dense_23 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 9         
Total params: 103,921
Trainable params: 103,921
Non-trainable params: 0
_________________________________________________________________


In [0]:
for i in range(0,len(x_train)):
  (a,b) = np.shape(x_train[i])
  x = np.reshape(x_train[i],(1,a,b))
  sepsis.fit(x=x,y=np.reshape(y_train[i],(-1,1)),validation)

1/1 [==============================] - 0s 234ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 1s 850ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 91ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 31ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 30ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 53ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 35ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 109ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 157ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 57ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 1/1
1/1 [=====================